In [2]:
import random, math, queue

In [3]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv('minute_weather.csv')

In [5]:
df = df.drop(columns=['rowID', 'hpwren_timestamp'])

In [7]:
def neighborhood_graph(p, n, d, eps, X, neighbors):
    
    distance = 0.0
    neighbor, low, high = 0, 0, 0
    eps_sq = eps ** 2
    
    i = 0
    while i < n:
        
        j = 0
        while j < (p * (n - 1) - 1):
            
            low = i + 1
            high = n - 1
            
            neighbor = random.randint(low, (high - low + 1))
            
            distance = 0.0
            k = 0
            while k < d:

                distance += np.power(X[i * d + k] - X[neighbor * d + k], 2)
                
                if distance > eps_sq:
                    break
                        
                k += 1
            
                if distance <= eps_sq:             
                    neighbors[i].append(neighbor)
                    neighbors[neighbor].append(i)
        
            j += 1
            
        i += 1
    
    return neighbors

In [8]:
def DBSCAN(n, core, neighbors, result):
    
    q = queue.Queue()
    neighbor, point, count = 0, 0, 0
    
    i = 0
    while i < n:
        
        q = queue.Queue()
        
        if core[i] == 0 and result[i] == -1:
            q.put(i)
            result[i] = count
            
            while not q.empty():
                
                point = q.get()
                
                j = 0
                while j < len(neighbors[point]):
                    
                    neighbor = neigbors[point][j]
                    
                    if core[neighbor] == -1 and result[neighbor] == -1:
                        q.put(neighbor)
                        result[neighbor] = count
                    
                    j += 1
            
            count += 1
        
        i += 1
    
    return result

In [9]:
def cluster(n, neighbors, core, result):
    
    neighbor = 0
    
    i = 0
    while i < n:
        
        if result[i] != -1:
            continue
        
        j = 0
        while j < len(neighbors[i]):
            
            neighbor = neighbors[i][j]
            
            if core[neighbor]:
                result[i] = result[neighbor]
                break
            
            j += 1
        
        i += 1
    
    return result

In [21]:
def sgb_dbscan(p, n, d, eps, minPts, X, result):
    
    neighbors = [[0] * n] * n
    core = [0] * n
    
    graph = neighborhood_graph(p, n, d, eps, X, neighbors)
    
    i = 0
    while i < n:
        
        core[i] = (len(graph[i]) >= (minPts * p))
        i += 1
    
    scan = DBSCAN(n, core, graph, result)
    
    clusters = cluster(n, graph, core, scan)
    
    return clusters

In [25]:
class SGB_DBSCAN:
    
    def __init__(self, p, eps, minPts):
        self.p = p
        self.eps = eps
        self.minPts = minPts
    
    def fit_predict(self, X):
        X = np.ascontiguousarray(X, dtype=np.float32)
        n, d = X.shape
        
        result = np.full(n, -1, dtype=np.int32)
        
        clusters = sgb_dbscan(self.p, n, d, self.eps, self.minPts, X, result)
        
        return clusters

In [26]:
scanner = SGB_DBSCAN(.005, 5, 10)

In [27]:
# clusters = scanner.fit_predict(df)
# print(clusters)